In [38]:
import time
def containsNonAscii(s):
    return any(ord(i)>127 for i in s)

def load_dataset(filename):
    words = []
    start_time = time.time()
    with open(filename, mode="r") as myFile:
        for line in myFile:
            try:
                words.append(line.strip())
            except:
                pass
    words = [word for word in words if  not containsNonAscii(word)]
    num_words = []
    for word in words:
        temp = []
        for char in word:
            c = ord(char)
            temp.append(c)
        num_words.append(temp)
    print "Reading the file takes %s seconds" % (time.time() - start_time)
    return num_words

In [39]:
def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    return map(frozenset, c1)

In [40]:
def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1

    num_items = float(len(dataset))
    retlist = []
    support_data = {}
    for key in sscnt:
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support
    return retlist, support_data


In [41]:
def aprioriGen(freq_sets, k):
    "Generate the joint transactions from candidate sets"
    retList = []
    lenLk = len(freq_sets)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            L1 = list(freq_sets[i])[:k - 2]
            L2 = list(freq_sets[j])[:k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(freq_sets[i] | freq_sets[j])
    return retList


In [42]:
def apriori(dataset, minsupport=0.5):
    "Generate a list of candidate item sets"
    C1 = createC1(dataset)
    D = map(set, dataset)
    L1, support_data = scanD(D, C1, minsupport)
    L = [L1]
    k = 2
    while (len(L[k - 2]) > 0):
        Ck = aprioriGen(L[k - 2], k)
        Lk, supK = scanD(D, Ck, minsupport)
        support_data.update(supK)
        L.append(Lk)
        k += 1

    return L, support_data

In [46]:
dataset = load_dataset("train.txt")
L,support_data = apriori(dataset,minsupport=0.05)
print len(support_data)

Reading the file takes 3.00740814209 seconds
988


In [53]:
def generateRules(L, support_data, min_confidence=0.15):
    rules = []
    for i in range(1, len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rules_from_conseq(freqSet, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freqSet, H1, support_data, rules, min_confidence)
    return rules


def calc_confidence(freqSet, H, support_data, rules, min_confidence=0.5):
    pruned_H = []
    for conseq in H:
        conf = support_data[freqSet] / support_data[freqSet - conseq]
        if conf >= min_confidence:
            rules.append((freqSet - conseq, conseq, conf))
            pruned_H.append(conseq)
    return pruned_H


def rules_from_conseq(freqSet, H, support_data, rules, min_confidence=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calc_confidence(freqSet, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            rules_from_conseq(freqSet, Hmp1, support_data, rules, min_confidence)

In [55]:
rules = generateRules(L,support_data)

In [77]:
seq_conf = {}
univ_key = []
univ_sub_key = []
univ_val = []
for item in rules:
    key = ""
    for i in item[0]:
        key += chr(i)
    univ_key.append(key)
    sub_key = ""
    for i in item[1]:
        sub_key += chr(i)
    univ_sub_key.append(sub_key)
    val = item[2]
    univ_val.append(val)

for i in xrange(len(univ_key)):
    if univ_key[i] in seq_conf:
        seq_conf[univ_key[i]][univ_sub_key[i]] = univ_val[i]
    else:
        seq_conf[univ_key[i]] = {}
        seq_conf[univ_key[i]][univ_sub_key[i]] = univ_val[i]
for key in seq_conf:
    print key,seq_conf[key]

en {'ai': 0.2354808791984114, 'is': 0.21870145831318172, 'ar': 0.2580127566841725, 'ir': 0.24629162849713515, 'it': 0.21230035675472686}
ae {'rt': 0.22134848484848485, 'rn': 0.22902020202020204, 'ir': 0.20951515151515152, 'in': 0.20902020202020205, 'rs': 0.2171767676767677}
ai {'tn': 0.19770132099410406, 're': 0.22114656744106703, 'rn': 0.2133154926273816, 'en': 0.22062414039427253}
is {'re': 0.2754039373075459, 'en': 0.26597239040930004}
ir {'an': 0.26245712074404937, 'te': 0.2546322714363485, 'ae': 0.2720922453316542, 'se': 0.26104694253687777, 'en': 0.28391150639827367}
it {'re': 0.31017649267743147, 'en': 0.29811203170316636, 'an': 0.29630635741165384}
an {'re': 0.23888569878255006, 'ir': 0.21080164152921405, 'ie': 0.21802415934844566, 'it': 0.19537147763969592}
as {'re': 0.26298536489899765}
ar {'te': 0.22525055250038548, 'en': 0.2330575114354731, 'ie': 0.21320861386647477, 'se': 0.22100529372462355, 'in': 0.20565863185485944}
at {'re': 0.302261426098471, 'in': 0.2557708090512218}